In [ ]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

## Getting Encounter quantities per patient and in general

In [ ]:
e = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\padded_matches_7_14.csv")
n = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\not_padded_matches_7_14.csv")

In [ ]:
e = e.drop(columns = ['Unnamed: 0.1.1.1.1'], axis = 1) #only run if runtime reset

In [ ]:
e.columns

In [ ]:
n["sequence_level_regex_match"] = n["merged_regex_sent"].to_list()

In [ ]:
n.columns = e.columns

In [ ]:
df = pd.concat([e, n])

In [ ]:
df

In [ ]:
speciality = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\specialty_visit_encounters.csv")
pcp = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\pcp_visit_encounters.csv")

In [ ]:
df = pd.merge(df, pcp, left_on = 'PatientEncounterID', right_on = 'PatientEncounterID', how = 'left')

In [ ]:
df = pd.merge(df, speciality, left_on = 'PatientEncounterID', right_on = 'PatientEncounterID', how = 'left')

In [ ]:
df.head()

In [ ]:
len(df["PatientID"].unique())

In [ ]:
df_clone = df.drop_duplicates(subset='PatientEncounterID', keep="first")

In [ ]:
len(df["PatientEncounterID"].unique())

In [ ]:
encounter_counts = df_clone.groupby(["EMPI"])['PatientEncounterID'].count().reset_index()
encounter_counts

In [ ]:
encounter_counts["PatientEncounterID"].describe()

In [ ]:
encounter_counts.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\encounter_counts.csv", index= False)

In [ ]:
speciality_counts = df.groupby(["EMPI"])['SpecialtyDSC'].count().reset_index()

In [ ]:
speciality_counts.head()

In [ ]:
speciality_counts["SpecialtyDSC"].describe()

In [ ]:
speciality_counts["SpecialtyDSC"].median()

In [ ]:
speciality_counts["SpecialtyDSC"].sum()

In [ ]:
pcp_counts = df[df["pcp_flag"] == 1].groupby(["EMPI"])["pcp_flag"].count().reset_index()

In [ ]:
pcp_counts.head()

In [ ]:
pcp_counts["pcp_flag"].describe()

In [ ]:
pcp_counts["pcp_flag"].median()

In [ ]:
pcp_counts["pcp_flag"].sum()

In [ ]:
speciality_counts.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\speciality_counts.csv", index = False)

In [ ]:
pcp_counts.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\pcp_counts.csv", index = False)

In [ ]:
df.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\MASTER_7_14.csv")

## Recalculating Keyword Matches

In [ ]:
unique_empi = df["PatientID"].unique().tolist()

In [ ]:
df

In [ ]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)

In [ ]:
l = []
for note in tqdm(df["padded_merged_regex_sent"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    #print(curr)
    #print(l)
    l.append(str(curr))

df["padded_regex_match"] = l

In [ ]:
cols =  ['dementia', 'cognition', 'memory', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's",  'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia', 'ad', 'lbd']
len(cols)

In [ ]:
summary_stats  = pd.DataFrame(pd.np.empty((len(unique_empi), len(cols))) * pd.np.nan) 
summary_stats.columns = cols

In [ ]:
df

In [ ]:
for i in tqdm(range(len(unique_empi))):
    keyword_list = list((df[df['PatientID'] == unique_empi[i]]['padded_regex_match'].values))
    #print("KL: ", keyword_list)
    match_tuples = []
    for j in range(len(keyword_list)):
        sub = "\', \'"
        x = list(eval(keyword_list[j]))# [2:len(keyword_list[j])-2]
        #print("X: ", x)
#         if sub in x:
#             #print("YES")
#             x = x.split("\', \'")
#             #print("X2: ", x)
#         else:
#             x = x.split("\", \"")
        for z in range(len(x)):
            match_tuples.append((x[z], 0))
    #print("MT: ", match_tuples)

    freq = {"dementia":0
        ,"cognition":0
        ,"cognition":0
        ,"memory":0
        ,"mmse":0
        ,"moca":0
        ,"alzheimer":0
        ,'cognitive impairment':0
        ,"mci":0
        ,"cerebellar":0
        ,"neurocognitive":0
        ,"lewy":0
        ,"pick's":0
        ,"corticobasal":0
        ,"cerebral":0
        ,"cerebrovascular":0
        ,"amnesia":0
        ,"ad": 0
        ,"lbd": 0
    }

    for k, v in match_tuples:   
        if (k != ''):
            if (k == 'cognitive impairmentcognitiveimpairment'):
                k = 'cognitive impairment'
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            else:
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            
    #print(freq.items())
    #print(summary_stats["dementia"])
    for k, v in freq.items():
        #print(str(k.lower()))
        summary_stats[str(k.lower())][i] = int(v)

In [ ]:
summary_stats

In [ ]:
summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\summary_stats_7_14.csv")

In [ ]:
summary_stats = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\summary_stats_7_14.csv")

In [ ]:
summary_stats["Sum"].describe()

In [ ]:
summary_stats["Sum"].median()

In [ ]:
match_counts = summary_stats.loc[[16428]].T
match_counts.columns = ['Sum']

In [ ]:
match_counts.sort_values(by = ['Sum'], ascending=False)

In [ ]:
match_counts = match_counts.drop(labels = ['Sum', 'Unnamed: 0'], axis = 0)

In [ ]:
match_counts

In [ ]:
match_counts.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\match_counts.csv")

In [ ]:
f1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\Biobank_filtered_csv\file1.csv")

In [ ]:
f2 = f1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\Biobank_filtered_csv\file2.csv")

In [ ]:
len(f1) + len(f2)

In [ ]:
empi = df[df["EMPI"].isin(df["EMPI"].unique().tolist())]

In [ ]:
empi

In [ ]:
empi = empi.drop_duplicates(subset = 'EMPI', keep = 'first')

In [ ]:
len(empi)

In [ ]:
dem = pd.concat([f1, f2])

In [ ]:
len(dem["EMPI"].unique())

In [ ]:
ages = pd.merge(empi, dem, left_on = 'EMPI', right_on = 'EMPI', how = 'left')

In [ ]:
ages

In [ ]:
ages = ages.drop_duplicates(subset = 'EMPI', keep = 'first')

In [ ]:
ages.columns

In [ ]:
sum(ages["Race"].value_counts().values.tolist())

In [ ]:
White-WHITE                                           6483
BLACK OR AFRICAN AMERICAN                              248
Other-OTHER                                            134
Not Recorded-DECLINED                                   75
Asian-ASIAN                                             67
Hispanic-HISPANIC OR LATINO                             53
Not Recorded-UNKNOWN                                    29
Not Recorded-@                                          20
Hispanic-OTHER@HISPANIC                                 18
Hispanic White-WHITE@HISPANIC                            5
American Indian-AMERICAN INDIAN OR ALASKA NATIVE         4
Hawaiian-NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER       3
Asian-ASIAN@HISPANIC                                     1
White-W                                                  1
Hispanic-@HISPANIC                                       1
Hispanic Black-BLACK OR AFRICAN AMERICAN@HISPANIC        1
Hispanic-HISPANIC OR LATINO@HISPANIC                     1

In [ ]:
dem1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\BioBank\sd587_03212118253982866_6375194824947826181_Dem.csv")
dem2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\BioBank\sd587_03212118253982866_6375194824947826182_Dem.csv")

In [ ]:
whole = pd.concat([dem1, dem2])
whole.head()

In [ ]:
len(whole["EMPI"].unique())

In [ ]:
patients = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\SLAT_production_7_24.csv")

In [ ]:
demographics = pd.DataFrame()

In [ ]:
race = []
for p in patients["EMPI"].unique():
    race.append(whole[whole["EMPI"] == p]["Race"].values)

In [ ]:
demographics["race"] = race
demographics["empi"] = patients["EMPI"].unique()

In [ ]:
demographics["race"].value_counts()

In [ ]:
sum(demographics["race"].value_counts())

In [ ]:
109+40+14+2+2+2+1

In [ ]:

[Not Recorded-UNKNOWN]                                     73
[White-W]                                                   1

##